# Generative Adversarial Networks (GANs) con CelebA Dataset
## Laboratorio 6 - Data Science Sección 20
~ Samuel Chamale 21885

~ Adrian Rodriguez 21691

In [1]:
import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers # type: ignore


In [2]:
# Constants
DATA_DIR = 'img_align_celeba/img_align_celeba'  # Replace with the path to your CelebA images | Download from https://www.kaggle.com/datasets/jessicali9530/celeba-dataset/
BATCH_SIZE = 128
IMG_HEIGHT = 64 # According to kaggle dataset
IMG_WIDTH = 64 # According to kaggle dataset
EPOCHS = 50
BUFFER_SIZE = 60000
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16
MODEL_SAVE_PATH = 'gan_generator.pkl'


In [5]:
# Configure TensorFlow to use the GPU and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to True to prevent TensorFlow from allocating all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using GPU: {gpus}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Using CPU instead.")

Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 1. Preparación de los datos

In [ ]:
def load_image(image_path):
    # Load image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize image
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    # Normalize image to [-1, 1]
    image = (image - 127.5) / 127.5
    return image

In [ ]:
image_paths = [os.path.join(DATA_DIR, fname) for fname in os.listdir(DATA_DIR)]
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)